In [3]:
# imports
import os
import pickle
import torch
import importlib
import constants
import utils
from simulator.nets import FeedForward
from simulator.parsing_funcs import parse_params
importlib.reload(constants)
importlib.reload(utils)

<module 'utils' from 'A:\\Dropbox\\eBay\\repo\\utils.py'>

### Attempting to load model

In [4]:
path = 'models'
os.chdir('A:/dropbox/ebay/') # BARRY'S Working directory

In [73]:
# loads a model given its name
def load_model(model_type, model_name):
    dir_path = '{}/{}/{}/'.format(path, model_type, model_name)
    info_path = '{}info.pkl'.format(dir_path)
    model_path = '{}1.pt'.format(dir_path )
    info = pickle.load(open(info_path, 'rb'))
    model = torch.load(model_path)
    return info, model
# gets types of everything in dict
def dict_types(input_dict):
    for key, value in input_dict.items():
        print('{}: {}'.format(key, type(value)))
        
        
def print_model_fixed(model_type, model_name):
    info, model = load_model(model_type, model_name)
    for info_element in info:
        for key, val in info_element.items():
            print('feature type: {}'.format(key))
            try:
                for col in val:
                    print(col)
            except:
                print(val)
    return set([el for el in info[0]['x_fixed']])
    

In [11]:
# arrival bin
ab_info, ab_model = load_model('arrival', 'bin')

In [13]:
ab_info

[{'x_fixed': Index(['start_days', 'holiday', 'dow0', 'dow1', 'dow2', 'dow3', 'dow4', 'dow5',
         'tf_slr_lstgs_open', 'tf_slr_lstgs',
         ...
         'focal_days', 'focal_holiday', 'focal_dow0', 'focal_dow1', 'focal_dow2',
         'focal_dow3', 'focal_dow4', 'focal_dow5', 'byr_us', 'byr_hist'],
        dtype='object', length=134)}, {'fixed': 134, 'out': 1}]

In [17]:
print(type(ab_info[0]))
print(type(ab_info[1]))

<class 'dict'>
<class 'dict'>


In [18]:
print(ab_info[0])

{'x_fixed': Index(['start_days', 'holiday', 'dow0', 'dow1', 'dow2', 'dow3', 'dow4', 'dow5',
       'tf_slr_lstgs_open', 'tf_slr_lstgs',
       ...
       'focal_days', 'focal_holiday', 'focal_dow0', 'focal_dow1', 'focal_dow2',
       'focal_dow3', 'focal_dow4', 'focal_dow5', 'byr_us', 'byr_hist'],
      dtype='object', length=134)}


In [25]:
for key, val in ab_info[0].items():
    print('key:{}'.format(key))
    for col in val:
        print(col)
        
for key, val in ab_info[1].items():
    print('key:{}'.format(key))
    print(val)

key:x_fixed
start_days
holiday
dow0
dow1
dow2
dow3
dow4
dow5
tf_slr_lstgs_open
tf_slr_lstgs
tf_slr_threads
tf_slr_slr_offers
tf_slr_byr_offers
tf_slr_accepts
tf_slr_accept_25
tf_slr_accept_50
tf_slr_accept_75
tf_slr_accept_100
tf_meta_lstgs_open
tf_meta_lstgs
tf_meta_threads
tf_meta_slr_offers
tf_meta_byr_offers
tf_meta_accepts
tf_meta_accept_25
tf_meta_accept_50
tf_meta_accept_75
tf_meta_accept_100
tf_leaf_lstgs_open
tf_leaf_lstgs
tf_leaf_threads
tf_leaf_slr_offers
tf_leaf_byr_offers
tf_leaf_accepts
tf_leaf_accept_25
tf_leaf_accept_50
tf_leaf_accept_75
tf_leaf_accept_100
tf_title_lstgs_open
tf_title_lstgs
tf_title_threads
tf_title_slr_offers
tf_title_byr_offers
tf_title_accepts
tf_title_accept_25
tf_title_accept_50
tf_title_accept_75
tf_title_accept_100
tf_cndtn_lstgs_open
tf_cndtn_lstgs
tf_cndtn_threads
tf_cndtn_slr_offers
tf_cndtn_byr_offers
tf_cndtn_accepts
tf_cndtn_accept_25
tf_cndtn_accept_50
tf_cndtn_accept_75
tf_cndtn_accept_100
relisted
store
slr_us
fast
photos
slr_bos
slr_lst

In [78]:
print_model_fixed('arrival', 'hist')
print_model_fixed('arrival', 'sec')
print_model_fixed('arrival', 'bin')

feature type: x_fixed
start_days
holiday
dow0
dow1
dow2
dow3
dow4
dow5
tf_slr_lstgs_open
tf_slr_lstgs
tf_slr_threads
tf_slr_slr_offers
tf_slr_byr_offers
tf_slr_accepts
tf_slr_accept_25
tf_slr_accept_50
tf_slr_accept_75
tf_slr_accept_100
tf_meta_lstgs_open
tf_meta_lstgs
tf_meta_threads
tf_meta_slr_offers
tf_meta_byr_offers
tf_meta_accepts
tf_meta_accept_25
tf_meta_accept_50
tf_meta_accept_75
tf_meta_accept_100
tf_leaf_lstgs_open
tf_leaf_lstgs
tf_leaf_threads
tf_leaf_slr_offers
tf_leaf_byr_offers
tf_leaf_accepts
tf_leaf_accept_25
tf_leaf_accept_50
tf_leaf_accept_75
tf_leaf_accept_100
tf_title_lstgs_open
tf_title_lstgs
tf_title_threads
tf_title_slr_offers
tf_title_byr_offers
tf_title_accepts
tf_title_accept_25
tf_title_accept_50
tf_title_accept_75
tf_title_accept_100
tf_cndtn_lstgs_open
tf_cndtn_lstgs
tf_cndtn_threads
tf_cndtn_slr_offers
tf_cndtn_byr_offers
tf_cndtn_accepts
tf_cndtn_accept_25
tf_cndtn_accept_50
tf_cndtn_accept_75
tf_cndtn_accept_100
relisted
store
slr_us
fast
photos
slr_b

{'accept',
 'accept_nines',
 'accept_round',
 'auto_dist',
 'byr_hist',
 'byr_us',
 'decline',
 'decline_nines',
 'decline_round',
 'dow0',
 'dow1',
 'dow2',
 'dow3',
 'dow4',
 'dow5',
 'fast',
 'fdbk_100',
 'fdbk_pstv',
 'fdbk_score',
 'focal_days',
 'focal_dow0',
 'focal_dow1',
 'focal_dow2',
 'focal_dow3',
 'focal_dow4',
 'focal_dow5',
 'focal_holiday',
 'has_accept',
 'has_decline',
 'holiday',
 'lda0',
 'lda1',
 'lda2',
 'lda3',
 'lda4',
 'lda5',
 'lda6',
 'meta1',
 'meta10',
 'meta11',
 'meta12',
 'meta13',
 'meta14',
 'meta15',
 'meta16',
 'meta17',
 'meta18',
 'meta2',
 'meta20',
 'meta21',
 'meta22',
 'meta23',
 'meta24',
 'meta25',
 'meta26',
 'meta27',
 'meta28',
 'meta29',
 'meta3',
 'meta30',
 'meta31',
 'meta32',
 'meta33',
 'meta34',
 'meta4',
 'meta5',
 'meta6',
 'meta7',
 'meta8',
 'meta9',
 'new',
 'photos',
 'refurb',
 'relisted',
 'slr_bos',
 'slr_lstgs',
 'slr_us',
 'start',
 'start_days',
 'start_nines',
 'start_round',
 'store',
 'tf_cndtn_accept_100',
 'tf_cndtn

In [64]:
ab.load_state_dict(torch.load('{}/{}_1.pt'.format(path, 'arrival_bin')))

RuntimeError: Error(s) in loading state_dict for FeedForward:
	Missing key(s) in state_dict: "seq.1.weight", "seq.1.bias". 
	Unexpected key(s) in state_dict: "seq.5.weight", "seq.5.bias". 
	size mismatch for seq.0.weight: copying a param with shape torch.Size([32, 134]) from checkpoint, the shape in current model is torch.Size([1, 134]).
	size mismatch for seq.0.bias: copying a param with shape torch.Size([32]) from checkpoint, the shape in current model is torch.Size([1]).
	size mismatch for seq.3.weight: copying a param with shape torch.Size([32, 32]) from checkpoint, the shape in current model is torch.Size([1, 1]).
	size mismatch for seq.3.bias: copying a param with shape torch.Size([32]) from checkpoint, the shape in current model is torch.Size([1]).

In [58]:
# bin exploration
sm_info, sm_model = load_model('slr', 'msg')

In [60]:
print(sm_info)

[{'x_fixed': Index(['byr_us', 'byr_hist', 'start_days', 'holiday', 'dow0', 'dow1', 'dow2',
       'dow3', 'dow4', 'dow5',
       ...
       'meta29', 'meta30', 'meta31', 'meta32', 'meta33', 'meta34', 'new',
       'used', 'refurb', 'wear'],
      dtype='object', length=126), 'x_time': Index(['con', 'norm', 'split', 'delay', 'days', 'holiday', 'dow0', 'dow1',
       'dow2', 'dow3', 'dow4', 'dow5', 'minutes', 'slr_offers', 'slr_best',
       'slr_offers_open', 'slr_best_open', 'byr_offers', 'byr_best',
       'byr_offers_open', 'byr_best_open', 'slr_offers_diff', 'slr_best_diff',
       'slr_offers_open_diff', 'slr_best_open_diff', 'byr_offers_diff',
       'byr_best_diff', 'byr_offers_open_diff', 'byr_best_open_diff',
       'con_other', 'norm_other', 'split_other', 'round_other', 'nines_other',
       'msg_other', 'delay_other', 'days_other', 'holiday_other', 'dow0_other',
       'dow1_other', 'dow2_other', 'dow3_other', 'dow4_other', 'dow5_other',
       'minutes_other', 'slr_offers_o

In [61]:
for col in sm_info[0]['x_fixed']:
    print(col)

byr_us
byr_hist
start_days
holiday
dow0
dow1
dow2
dow3
dow4
dow5
tf_slr_lstgs_open
tf_slr_lstgs
tf_slr_threads
tf_slr_slr_offers
tf_slr_byr_offers
tf_slr_accepts
tf_slr_accept_25
tf_slr_accept_50
tf_slr_accept_75
tf_slr_accept_100
tf_meta_lstgs_open
tf_meta_lstgs
tf_meta_threads
tf_meta_slr_offers
tf_meta_byr_offers
tf_meta_accepts
tf_meta_accept_25
tf_meta_accept_50
tf_meta_accept_75
tf_meta_accept_100
tf_leaf_lstgs_open
tf_leaf_lstgs
tf_leaf_threads
tf_leaf_slr_offers
tf_leaf_byr_offers
tf_leaf_accepts
tf_leaf_accept_25
tf_leaf_accept_50
tf_leaf_accept_75
tf_leaf_accept_100
tf_title_lstgs_open
tf_title_lstgs
tf_title_threads
tf_title_slr_offers
tf_title_byr_offers
tf_title_accepts
tf_title_accept_25
tf_title_accept_50
tf_title_accept_75
tf_title_accept_100
tf_cndtn_lstgs_open
tf_cndtn_lstgs
tf_cndtn_threads
tf_cndtn_slr_offers
tf_cndtn_byr_offers
tf_cndtn_accepts
tf_cndtn_accept_25
tf_cndtn_accept_50
tf_cndtn_accept_75
tf_cndtn_accept_100
relisted
store
slr_us
fast
photos
slr_bos
slr

In [64]:
for col in sm_info[0]['x_time']:
    print(col)

con
norm
split
delay
days
holiday
dow0
dow1
dow2
dow3
dow4
dow5
minutes
slr_offers
slr_best
slr_offers_open
slr_best_open
byr_offers
byr_best
byr_offers_open
byr_best_open
slr_offers_diff
slr_best_diff
slr_offers_open_diff
slr_best_open_diff
byr_offers_diff
byr_best_diff
byr_offers_open_diff
byr_best_open_diff
con_other
norm_other
split_other
round_other
nines_other
msg_other
delay_other
days_other
holiday_other
dow0_other
dow1_other
dow2_other
dow3_other
dow4_other
dow5_other
minutes_other
slr_offers_other
slr_best_other
slr_offers_open_other
slr_best_open_other
byr_offers_other
byr_best_other
byr_offers_open_other
byr_best_open_other
slr_offers_diff_other
slr_best_diff_other
slr_offers_open_diff_other
slr_best_open_diff_other
byr_offers_diff_other
byr_best_diff_other
byr_offers_open_diff_other
byr_best_open_diff_other
reject_last
round_last
nines_last
msg_last
auto_last
exp_last
t1
t2


### Exploring RL Inputs

In [32]:
data_path = 'data/partitions/train_rl'
x_path = '{}/x_lstg_cols.pkl'.format(data_path)

In [33]:
x = utils.unpickle(x_path)

In [38]:
print(type(x))
print(len(x))

<class 'dict'>
124


In [47]:
x_lstg = [el for el in x]
ab_info_inputs = [el for el in ab_info[0]['x_fixed']]
x_lstg = set(x_lstg)
ab_info_inputs = set(ab_info_inputs)

In [56]:
# elements in arrival bin model that aren't in the x_lstg object
# significant that it includes dow0,...and focal_dow0
print(ab_info_inputs.difference(x_lstg))

{'byr_us', 'byr_hist', 'focal_dow2', 'focal_holiday', 'focal_dow1', 'focal_dow3', 'focal_dow0', 'focal_dow5', 'focal_days', 'focal_dow4'}


In [53]:
days = [el for el in ab_info_inputs if 'days' in el]
days

['focal_days', 'start_days']

In [55]:
# elements in x_lstg object that aren't in arrival bin model
print(x_lstg.difference(ab_info_inputs))

set()


In [36]:
for el in x:
    print(el)

start_days
holiday
dow0
dow1
dow2
dow3
dow4
dow5
tf_slr_lstgs_open
tf_slr_lstgs
tf_slr_threads
tf_slr_slr_offers
tf_slr_byr_offers
tf_slr_accepts
tf_slr_accept_25
tf_slr_accept_50
tf_slr_accept_75
tf_slr_accept_100
tf_meta_lstgs_open
tf_meta_lstgs
tf_meta_threads
tf_meta_slr_offers
tf_meta_byr_offers
tf_meta_accepts
tf_meta_accept_25
tf_meta_accept_50
tf_meta_accept_75
tf_meta_accept_100
tf_leaf_lstgs_open
tf_leaf_lstgs
tf_leaf_threads
tf_leaf_slr_offers
tf_leaf_byr_offers
tf_leaf_accepts
tf_leaf_accept_25
tf_leaf_accept_50
tf_leaf_accept_75
tf_leaf_accept_100
tf_title_lstgs_open
tf_title_lstgs
tf_title_threads
tf_title_slr_offers
tf_title_byr_offers
tf_title_accepts
tf_title_accept_25
tf_title_accept_50
tf_title_accept_75
tf_title_accept_100
tf_cndtn_lstgs_open
tf_cndtn_lstgs
tf_cndtn_threads
tf_cndtn_slr_offers
tf_cndtn_byr_offers
tf_cndtn_accepts
tf_cndtn_accept_25
tf_cndtn_accept_50
tf_cndtn_accept_75
tf_cndtn_accept_100
relisted
store
slr_us
fast
photos
slr_bos
slr_lstgs
fdbk_scor